## Import Dependencies

In [41]:
import pandas as pd
import requests
import pymongo
import os
import tweepy
import json
import numpy as np

from datetime import datetime
from splinter import Browser
from bs4 import BeautifulSoup as bs
from pprint import pprint

consumer_key = "36Ir92uhNmDnv6Sn6tr2BicdY"
consumer_secret = "URccAlZmOS3COMVspRM92dla9lNi76NpbUMhrdEUOmH5OWLcvO"
access_token = "1038858933560115200-NgoqDCmZUVm9Z41KWN89s1MrB2tegs"
access_token_secret = "l3kG1sk2mCWWTaZIdC7RDoJZxN1iWZ3JdYXYf3XyItqfR"

# Setup Tweepy Authentication 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

## Step 1 - Scraping

In [18]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [19]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [20]:
db = client.space_db
collection = db.articles

## NASA Mars News

In [21]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

In [22]:
# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')

In [23]:
results = soup.find_all("div", class_="list_text")

In [24]:
html = browser.html
soup = bs(html, 'html.parser')

posts = soup.find_all("div", class_="list_text")

title = []
para = []

for post in posts:
    title.append(post.a.text)
    para.append(post.find("div", class_="article_teaser_body").text)

    article ={
        "news_title": title[0],
        "news_text": para[0]
    }

#     news_title = title[0]
#     news_p = para[0]

In [25]:
print(article["news_title"])
print(article["news_text"])

NASA Hears MarCO CubeSats Loud and Clear from Mars 
A pair of tiny, experimental spacecraft fulfilled their mission yesterday, relaying back near-real-time data during InSight's landing.


In [26]:
for result in results:
    title = result.a.text
    para = result.find("div", class_="article_teaser_body").text
    
    article = {
        "news_title": title,
        "news_paragraph": para
    }
    
    print("---------------------------------")
    print(f'{title}')
    print(f'{para}')

## Finding the Images URL

In [27]:
url2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url2)

In [28]:
# Retrieve page with the requests module
response_image = requests.get(url2)

# Create BeautifulSoup object; parse with 'lxml'
soup_image = bs(response_image.text, 'lxml')

In [29]:
browser.click_link_by_partial_text('FULL IMAGE')

In [30]:
browser.click_link_by_partial_text('more info')

ElementNotVisibleException: Message: element not visible
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.34.522940 (1a76f96f66e3ca7b8e57d503b4dd3bccfba87af1),platform=Windows NT 10.0.17134 x86_64)


In [31]:
# URL varies depending on what the web browser opens
url_image = "https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA18185"

In [32]:
# Retrieve page with the requests module
response_image1 = requests.get(url_image)

# Create BeautifulSoup object; parse with 'lxml'
soup_image1 = bs(response_image1.text, 'lxml')

In [33]:
results_image = soup_image1.find_all("figure", class_="lede")

In [34]:
# Scarping the image url
image_url = []
for image in results_image:
    image_url.append(image.a["href"])

In [35]:
image_url

['/spaceimages/images/largesize/PIA18185_hires.jpg']

In [36]:
# Adding the url into images
for url in image_url:
    featured_image_url= "https://www.jpl.nasa.gov"+url
    print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18185_hires.jpg


## Mars Weather

In [42]:
target_user = "@MarsWxReport"

In [43]:
public_tweets = api.user_timeline(target_user)

In [44]:
weather_tweet = []

for tweet in public_tweets:
    weather_tweet.append(tweet["text"])

In [45]:
mars_weather = weather_tweet[0]
print(mars_weather)

Sol 2240 (2018-11-24), high 0C/32F, low -70C/-93F, pressure at 8.49 hPa, daylight 06:28-18:44


## Mars Facts

In [ ]:
url_facts = "https://space-facts.com/mars/"

In [ ]:
tables = pd.read_html(url_facts)
tables

In [ ]:
df = tables[0]

In [ ]:
df_clean = df.set_index(0)
df_clean.index.name=None

In [ ]:
df_clean = df_clean.rename(columns={1: "Values"})
df_clean

In [ ]:
html_table = df_clean.to_html()

In [ ]:
html_table.replace('\n', '')

In [ ]:
df_clean.to_html('mars_facts.html')

## Mars Hemisphere

In [ ]:
url_cerberus = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
url_schiaparelli = "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced"
url_syrtis = "https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced"
url_valles = "https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"

In [ ]:
url_hemisphere = [url_cerberus, url_schiaparelli, url_syrtis, url_valles]

for url in url_hemisphere:
    response_hemisphere = requests.get(url)
    soup_hemisphere = bs(response_hemisphere.text, 'lxml')
    
    results_hemisphere = soup_hemisphere.find_all("div", class_="container")
    
    for result in results_hemisphere:
        title = result.h2.text
        img_url = result.find("img", class_="wide-image")["src"]
        
        hemisphere_image_urls = {
            "title": title,
            "img_url": "https://astrogeology.usgs.gov"+img_url
        }
        
        print(hemisphere_image_urls)

In [ ]:
print(hemisphere_image_urls["img_url"])